In [23]:
import torch
import cv2 as cv
import numpy as np
import onnxruntime
import onnx


In [13]:
def prediction(img_pth, model):
  cls= ['Pizza', 'Steak']
  img= cv.imread(img_pth)
  img= cv.resize(img, (224, 224))
  a= img.transpose(2, 0, 1)
  a= np.expand_dims(a, 0)
  a= a/255
  a= torch.from_numpy(a)
  a= a.type(torch.float)
  pred=model(a)
  print(f'Raw prediction: {pred}')
  print(f'Model Predicted: {cls[pred.argmax()]}')
  return
  #  cv2_imshow(img)

* Load the PyTorch model which was saved using TorchScript
* Always save PyTorch model after training using
```
model_scripted = torch.jit.script(model)
model_scripted.save('model_scripted.pt')
```



In [ ]:
model = torch.jit.load('model_scripted.pt')
model.eval()

In [24]:
img= cv.imread('/content/download.jpg')
img= cv.resize(img, (224, 224))
a= img.transpose(2, 0, 1)
a= np.expand_dims(a, 0)
a= a/255
a= torch.from_numpy(a)
a= a.type(torch.float)
pred=model(a)
# prediction('/content/meat-steak_131550-9.avif', model)

In [25]:
pred

tensor([[0.6839, 0.3161]], grad_fn=<SoftmaxBackward0>)

* Exporting a PyTorch model to ONNX and saving in ONNX format.

In [ ]:
# specify the input shape of the onnx model
x = torch.randn(1, 3, 224, 224, requires_grad=True)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# x = x.to(device)

In [12]:
import torch.onnx
torch.onnx.export(model,
                  x,
                  "model.onnx",
                  export_params=True
                  )

* Cheaking and printing the onnx model 

In [ ]:
import onnx

# Load the ONNX model
model = onnx.load("/content/model.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

* Inferencing on the ONNX model using ONNX Runtime

* The input for inferencing is a NumPy array and has the same data type as the model parameters.

In [ ]:
img= cv.imread('/content/pizza_steak/test/pizza/1179703.jpg')
img= cv.resize(img, (224, 224))
img= img.transpose(2, 0, 1)
img= np.expand_dims(img, 0)
img= img/255
img= img.astype('float32')

In [ ]:
session = onnxruntime.InferenceSession('/content/model.onnx')
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(input_name)
print(output_name)

In [ ]:
# inferencing
result = session.run([output_name], {input_name: img})
result

* Converting ONNX to TFLite using OpenVINO.

In [29]:
!pip install -q --upgrade silabs-mltk
!pip install -q onnx onnx_tf
!pip install -q openvino_dev
!pip install -q onnx-simplifier onnxruntime
!pip install -q openvino2tensorflow 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 72.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 74.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 KB 21.7 MB/s eta 0

In [42]:
from openvino.tools.mo import main as mo_main
import onnx
import onnxsim
import onnx
from onnx_tf.backend import prepare
from mltk.utils.shell_cmd import run_shell_cmd
import sys

In [31]:
# Load the ONNX model
model_path = '/content/cat_vs_dog.onnx'
onnx_model = onnx.load(model_path)
tf_rep = prepare(onnx_model)

# Get the input tensor shape
input_tensor = tf_rep.signatures[tf_rep.inputs[0]]
input_shape = input_tensor.shape
input_shape_str = '[' + ','.join([str(x) for x in input_shape]) + ']'  # it return the input shape in the string format


In [44]:
openvino_out_dir = './openvino'
# os.makedirs(openvino_out_dir, exist_ok=True)

In [45]:
# print(f'Generating openvino at: {openvino_out_dir}')
cmd = [ 
    sys.executable, mo_main.__file__, 
    '--input_model', model_path,
    '--input_shape', input_shape_str,
    '--output_dir', openvino_out_dir,
    '--data_type', 'FP32'

]
retcode, retmsg = run_shell_cmd(cmd,  outfile=sys.stdout)
assert retcode == 0, 'Failed to do conversion' 

[ WARNING ]  Use of deprecated cli option --data_type detected. Option use in the following releases will be fatal. 
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /content/openvino22/cat_vs_dog.xml
[ SUCCESS ] BIN file: /content/openvino22/cat_vs_dog.bin


In [47]:
openvino2tensorflow_out_dir = './openvino2tensorflow'
openvino_xml_name = os.path.basename(model_path)[:-len('.onnx')] + '.xml'

In [51]:
# path= model_path + '/' + openvino_xml_name

In [53]:
# path

'/content/cat_vs_dog.onnx/cat_vs_dog.xml'

In [ ]:
import os 
from mltk.utils.shell_cmd import run_shell_cmd
# openvino_xml_name = os.path.basename(simplified_onnx_model_path)[:-len('.onnx')] + '.xml'
# openvinio_xml_name= 'cat_vs_dog.xml'


if os.name == 'nt':
  openvino2tensorflow_exe_cmd = [sys.executable, os.path.join(os.path.dirname(sys.executable), 'openvino2tensorflow')]
else:
  openvino2tensorflow_exe_cmd = ['openvino2tensorflow']

print(f'Generating openvino2tensorflow model at: {openvino2tensorflow_out_dir} ...')
cmd = openvino2tensorflow_exe_cmd + [ 
    '--model_path', '/content/openvino/cat_vs_dog.xml',
    # '--model_path', path,
    '--model_output_path', openvino2tensorflow_out_dir,
    # '--output_saved_model',
    '--output_no_quant_float32_tflite'
]

retcode, retmsg = run_shell_cmd(cmd)
assert retcode == 0, retmsg
print('done')

Generating openvino2tensorflow model at: ./openvino2tensorflow ...
